<a href="https://colab.research.google.com/github/park-geun-hyeong/Computer_Vision/blob/main/papers/EfficientNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=cff44800f39d7481dbd71a8272b74f7b835a039106c2c89017f20900fb56da84
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch


In [ ]:
from efficientnet_pytorch import EfficientNet

In [ ]:
model = EfficientNet.from_pretrained("efficientnet-b0")

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth


  0%|          | 0.00/20.4M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b0


In [ ]:
model

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F

import numpy as np
import os

## Define Swish activation function

In [49]:
class Swish(nn.Module):
    def __init__(self):
        super().__init__()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return x * self.sigmoid(x)


if __name__ == '__main__':
    x = torch.randn(3,3,224,224)
    model = Swish()
    output = model(x)
    print(f"output_size: {output.size()}")  

output_size: torch.Size([3, 3, 224, 224])


In [50]:
class SEBlock(nn.Module):
    def __init__(self, in_channels, r=4):
        super().__init__()

        self.squeeze = nn.AdaptiveAvgPool2d((1,1))
        self.excitation = nn.Sequential(
            nn.Linear(in_channels, in_channels * r),
            Swish(), ## x * sigmoid(x)
            nn.Linear(in_channels * r, in_channels),
            nn.Sigmoid()    
        )

    def forward(self, x ):
        x = self.squeeze(x)
        x = x.view(x.size(0), -1)    
        x = self.excitation(x)
        x = x.view(x.size(0), x.size(1), 1,1)
        return x  


if __name__ == '__main__':
    x = torch.randn(3,56,17,17)
    model = SEBlock(x.size(1))
    output = model(x)
    print(f"model_size:{output.size()}")

model_size:torch.Size([3, 56, 1, 1])


In [69]:
class MBConv(nn.Module):
    expand = 6
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, se_scale=4, p=0.5):
        super().__init__()

        ## 첫번쨰 MBConv는 stochastic depth를 사용하지 않는다
        self.p = torch.tensor(p).float() if (in_channels == out_channels) else torch.tensor(1).float()
        self.residual = nn.Sequential(
            nn.Conv2d(in_channels, in_channels * MBConv.expand, 1 , stride = stride, padding=0, bias=False ),
            nn.BatchNorm2d(in_channels * MBConv.expand, momentum=0.99, eps=1e-3),
            nn.Conv2d(in_channels * MBConv.expand, in_channels * MBConv.expand, kernel_size = kernel_size, stride=1, padding = kernel_size //2, bias=False, groups = in_channels * MBConv.expand), ## input_channel만큼의 groups 만들어주기
            nn.BatchNorm2d(in_channels * MBConv.expand, momentum=0.99, eps=1e-3),
            Swish()

        )

        self.se = SEBlock(in_channels * MBConv.expand, se_scale)
        self.project = nn.Sequential(
            nn.Conv2d(in_channels * MBConv.expand, out_channels, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(out_channels, momentum=0.99, eps=-1e-3)
        )

        self.shortcut = (stride==1) and (in_channels==out_channels)

        def forward(self, x ):

            if self.training:
                if not torch.bernoulli(self.p):
                    return x 

                x_shortcut = x
                x_residual = self.residual(x)
                x_se = self.se(x_residual)
                
                x = x_se * x_residual
                x = self.project(x)

                if self.shortcut:
                    x = x_shortcut + x 

                return x

In [70]:
class SepConv(nn.Module):
    expand = 1
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, se_scale=4, p=0.5):
        super().__init__()

        ## 첫번쨰 MBConv는 stochastic depth를 사용하지 않는다
        self.p = torch.tensor(p).float() if (in_channels == out_channels) else torch.tensor(1).float()
        self.residual = nn.Sequential(
           nn.Conv2d(in_channels*SepConv.expand, in_channels*SepConv.expand, kernel_size=kernel_size, stride=1, padding=kernel_size//2, bias=False, groups = in_channels*SepConv.expand),
           nn.BatchNorm2d(in_channels*SepConv.expand, momentum=0.99, eps=1e-3),
           Swish()

        )

        self.se = SEBlock(in_channels * MBConv.expand, se_scale)
        self.project = nn.Sequential(
            nn.Conv2d(in_channels * SepConv.expand, out_channels, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(out_channels, momentum=0.99, eps=-1e-3)
        )

        self.shortcut = (stride==1) and (in_channels==out_channels)

        def forward(self, x ):

            if self.training:
                if not torch.bernoulli(self.p):
                    return x 

                x_shortcut = x
                x_residual = self.residual(x)
                x_se = self.se(x_residual)
                
                x = x_se * x_residual
                x = self.project(x)

                if self.shortcut:
                    x = x_shortcut + x 

                return x

In [71]:
class EfficientNet(nn.Module):
    def __init__(self, num_classes=10, width_coef=1., depth_coef=1., scale=1., dropout=0.2, se_scale=4, stochastic_depth=False, p=0.5):
        super(EfficientNet, self).__init__()

        channels = [32, 16, 24, 24, 40, 80, 112, 192, 320, 1280] ## width
        repeats = [1,2,2,3,3,4,1] ## depth
        strides = [1,2,2,2,1,2,1] 
        kernel_size = [3,3,5,3,5,5,3]
        depth = depth_coef
        width = width_coef

        channels = [int(x*width) for x in channels]
        repeats = [int(x*depth) for x in repeats]

        if stochastic_depth:
            self.p=p
            self.step = (1-0.5) / (sum(repeats)-1) 
        else:
            self.p=1
            self.step=0


        self.upsample = nn.Upsample(scale_factor = scale, mode='bilinear', align_corners=False) ## 4-dimention image_shaep scaling

        self.stage1 = nn.Sequential(
            nn.Conv2d(3, channels[0], 3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(channels[0], momentum=0.99, eps=1e-3)

        ) ## 3 => 32 (w,h일정)

        self.stage2 = self._make_Block(SepConv, repeats[0], channels[0], channels[1], kernel_size[0], strides[0], se_scale)
        self.stage3 = self._make_Block(MBConv, repeats[1], channels[1], channels[2], kernel_size[1], strides[1], se_scale)
        self.stage4 = self._make_Block(MBConv, repeats[2], channels[2], channels[3], kernel_size[2], strides[2], se_scale)
        self.stage5 = self._make_Block(MBConv, repeats[3], channels[3], channels[4], kernel_size[3], strides[3], se_scale)
        self.stage6 = self._make_Block(MBConv, repeats[4], channels[4], channels[5], kernel_size[4], strides[4], se_scale)
        self.stage7 = self._make_Block(MBConv, repeats[5], channels[5], channels[6], kernel_size[5], strides[5], se_scale)
        self.stage8 = self._make_Block(MBConv, repeats[6], channels[6], channels[7], kernel_size[6], strides[6], se_scale)
        self.stage9 = nn.Sequential(
            nn.Conv2d(channels[7], channels[8], 1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(channels[8], momentum=0.99, eps=1e-3),
            Swish()
        )

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.dropout = nn.Dropout(p=dropout)
        self.linear = nn.Linear(channels[8], num_classes)


    def forward(self, x):
        x = self.upsample(x)
        x = self.stage1(x)
        x = self.stage2(x)
        x = self.stage3(x)
        x = self.stage4(x)
        x = self.stage5(x)
        x = self.stage6(x)
        x = self.stage7(x)
        x = self.stage8(x)
        x = self.stage9(x)
        x = self.avgpoo(x)
        x = x.view(x.size(0), -1)
        x = x.dropout(x)
        x = x.linear(x)

        return x



    def _make_Block(self, block, repeats, in_channels, out_channels, kernel_size, stride, se_scale) :

        strides = [stride] + [1] * (repeats-1)
        layers = []
        for stride in strides:
            layers.append(block(in_channels, out_channels, kernel_size, stride, se_scale, self.p))
            in_channels = out_channels
            self.p -= self.step

        return nn.Sequential(*layers)             

        

In [72]:
def efficientnet_b0(num_classes=10):
    return EfficientNet(num_classes = num_classes, width_coef=1.0, depth_coef=1.0, scale=1.0)

def efficientnet_b1(num_classes=10):
    return EfficientNet(num_classes=num_classes, width_coef=1.0, depth_coef=1.1, scale=240/224, dropout=0.2, se_scale=4)

def efficientnet_b2(num_classes=10):
    return EfficientNet(num_classes=num_classes, width_coef=1.1, depth_coef=1.2, scale=260/224., dropout=0.3, se_scale=4)

def efficientnet_b3(num_classes=10):
    return EfficientNet(num_classes=num_classes, width_coef=1.2, depth_coef=1.4, scale=300/224, dropout=0.3, se_scale=4)

def efficientnet_b4(num_classes=10):
    return EfficientNet(num_classes=num_classes, width_coef=1.4, depth_coef=1.8, scale=380/224, dropout=0.4, se_scale=4)

def efficientnet_b5(num_classes=10):
    return EfficientNet(num_classes=num_classes, width_coef=1.6, depth_coef=2.2, scale=456/224, dropout=0.4, se_scale=4)

def efficientnet_b6(num_classes=10):
    return EfficientNet(num_classes=num_classes, width_coef=1.8, depth_coef=2.6, scale=528/224, dropout=0.5, se_scale=4)

def efficientnet_b7(num_classes=10):
    return EfficientNet(num_classes=num_classes, width_coef=2.0, depth_coef=3.1, scale=600/224, dropout=0.5, se_scale=4)

In [73]:
efficientnet_b0()

EfficientNet(
  (upsample): Upsample(scale_factor=1.0, mode=bilinear)
  (stage1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
  )
  (stage2): Sequential(
    (0): SepConv(
      (residual): Sequential(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (1): BatchNorm2d(32, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
        (2): Swish(
          (sigmoid): Sigmoid()
        )
      )
      (se): SEBlock(
        (squeeze): AdaptiveAvgPool2d(output_size=(1, 1))
        (excitation): Sequential(
          (0): Linear(in_features=192, out_features=768, bias=True)
          (1): Swish(
            (sigmoid): Sigmoid()
          )
          (2): Linear(in_features=768, out_features=192, bias=True)
          (3): Sigmoid()
        )
      )
      (project): Sequential(
  